# 要約 
このJupyter Notebookでは、LMSYS - Chatbot Arenaのコンペティションにおける「人間による好み予測」のための機械学習モデルを構築しています。具体的には、ユーザーが好むチャットボットの応答を予測することを目的としています。ノートブックはPythonを使用し、さまざまなデータ処理や深層学習技術を活用しています。

### 主な内容と手法

1. **ライブラリのインポート**:
   - データ処理には`pandas`と`numpy`を使用。
   - トークナイゼーションやモデルには、`transformers`ライブラリの各種トークナイザーとモデル（特にDeBERTa V2）を利用。
   - モデルの構築には`TensorFlow`とそのKeras APIを使用。

2. **データの読み込み**:
   - トレーニングデータ（`train.csv`）とテストデータ（`test.csv`）を読み込み、それぞれのデータフレームを作成。

3. **データの前処理**:
   - テキストデータの整形や結合を行う`combine_text`関数を定義し、プロンプトおよびレスポンスを処理。
   - ラベルを生成するための`create_label`関数も定義し、互換性のある形式でラベルを作成。

4. **トークナイゼーション**:
   - `DebertaV2Tokenizer`を用いて、プロンプトとレスポンスに対するトークンを生成し、必要なトークン型、注意マスクを作成。

5. **モデルの構築**:
   - CNN、LSTM、またはそのハイブリッドモデルとしてCNN-LSTMモデルを構築。モデルは、埋め込み層、畳み込み層、LSTM層を含み、最終的な出力層は3クラスのラベルに対応。

6. **モデルのトレーニング**:
   - モデルを訓練（フィッティング）するための設定を行い、早期終了の仕組みを導入。トレーニングは、トレーニングデータからバリデーションを使用して行われます。

7. **テストと予測**:
   - テストデータに対して同様の前処理を行い、モデルを用いて予測を実施。
   - 予測結果はデータフレームに変換され、最終的に`submission.csv`として保存されます。

全体を通じて、このノートブックは特に大規模言語モデルを活用したコンペティションに適したデータ処理とモデル設計に焦点を当てています。モデルの選択やトレーニング方法は、ユーザーの応答の好みを正確に予測するために設計されています。

---


# 用語概説 
以下に、Jupyter Notebookに登場する専門用語について、初心者がつまずきやすいマイナーなものや実務経験がないと馴染みが薄い言葉の簡単な解説を示します。

1. **前処理 (Preprocessing)**: データ分析や機械学習モデルのトレーニングの前に、データを整理・変換するステップのこと。ノイズを除去したり、形式を統一したりすることで、モデルの性能を向上させる。

2. **トークナイザー (Tokenizer)**: テキストデータを処理するためのツールで、文や単語をトークンに分割する役割を持つ。トークン化は、自然言語処理において非常に重要で、モデルがテキストを理解できる形に変える。

3. **入力ID (Input IDs)**: トークナイザーによって変換されたテキストのトークンが数値化されたもので、モデルに与える際の入力データ。各トークンは語彙のインデックスによって表現される。

4. **アテンションマスク (Attention Mask)**: 入力シーケンスにおいて、各トークンがモデルに対して無視される（パディングなどの場合）か考慮されるかを示すバイナリマスク。1はトークンが有効、0は無効を意味する。

5. **グローバルマックスプーリング (Global Max Pooling)**: 畳み込みニューラルネットワークにおいて、各フィルターの出力の中で最大値を取得する操作のこと。これにより、異なる特徴を集約し、次の層へ送る。

6. **カテゴリカルラベル (Categorical Labels)**: クラス分類問題において、各データがどのクラスに属するかを示すラベル。通常、数値の配列で表現され、複数のクラスの中から正しいクラスを予測するために使用される。

7. **ドロップアウト (Dropout)**: ニューラルネットワークの過学習を防ぐために、各エポックの間にランダムに一定の割合のユニット（ノード）を無効にする手法。これにより、モデルの一般化能力が向上する。

8. **エポック (Epoch)**: モデルが全ての訓練データを1回処理することを指す。訓練プロセスにおいてエポック数を指定することで、モデルが何回データを学習するかを制御する。

9. **フィッティング (Fitting)**: モデルがトレーニングデータに対して学習するプロセス。モデルのパラメータを調整し、データのパターンを学ぶ。

10. **ハイパーパラメータ (Hyperparameters)**: モデルの学習プロセスに関わる設定で、自動的に学習されないパラメータ。例としてはエポック数、バッチサイズ、学習率などがある。

これらの用語は、機械学習や深層学習の初心者が実務において理解し、使いこなすために重要です。また、具体的な文脈やプロジェクトにおいてどう定義され、どう使われるかを知ることが理解を深める助けとなります。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これは、kaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、以下のいくつかの便利なパッケージをロードします

import numpy as np # 線形代数ライブラリ
import pandas as pd # データ処理ライブラリ、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは、読み取り専用の"../input/"ディレクトリで利用できます
# 例えば、これを実行すると（実行ボタンをクリックするかShift + Enterを押すことで）、入力ディレクトリ内のすべてのファイルがリストされます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # 各ファイルのパスを表示します
        print(os.path.join(dirname, filename))

# 現在のディレクトリ（/kaggle/working/）に最大20GBまで書き込むことができます
# これは、「Save & Run All」を使用してバージョンを作成する際に出力として保存されます
# また、一時ファイルを/kaggle/temp/に書き込むこともできますが、これらは現在のセッションの外部に保存されません

## ライブラリのインポート

In [ ]:
import pandas as pd  # データを処理するためのpandasライブラリをインポートします
import numpy as np  # 数値計算のためのnumpyライブラリをインポートします
from datasets import Dataset  # datasetsライブラリからDatasetクラスをインポートします（データセットの扱いに便利です）
from functools import partial  # 関数を部分的に適用するためのpartial関数をインポートします
from sklearn.model_selection import train_test_split  # データを訓練用とテスト用に分割するためのtrain_test_split関数をインポートします
from transformers import AutoTokenizer, TFAutoModel  # Hugging Faceのトランスフォーマーモデルとトークナイザーをインポートします
from transformers import DebertaV2Tokenizer  # DeBERTa V2用のトークナイザーをインポートします
import tensorflow as tf  # 深層学習ライブラリのTensorFlowをインポートします
from tensorflow.keras.models import Sequential  # Kerasの順次モデルをインポートします
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input  # Kerasのレイヤーをインポートします
from keras.preprocessing import sequence as sq  # シーケンス処理のためのKerasのpreprocessingモジュールをインポートします

In [ ]:
# train.csvファイルを読み込み、データをtrainデータフレームに格納します
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')

# データフレームの最初の5行を表示します
train.head(5)  # データの最初の5行を表示して、内容を確認します

In [ ]:
# test.csvファイルを読み込み、データをtestデータフレームに格納します
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# データフレームの最初の5行を表示します
test.head(5)  # データの最初の5行を表示して、内容を確認します

## 前処理

In [ ]:
def combine_text(df):
    def process(input_str):
        # 文字列の前後のブラケットを取り除きます
        stripped_str = input_str.strip('[]')
        # 文字列を分割し、各文の前後の引用符を取り除きます
        sentences = [s.strip('"') for s in stripped_str.split('","')]
        # 文をスペースで結合して返します
        return  ' '.join(sentences)

    # 配列を単一の文字列に変換します
    df['prompt'] = df['prompt'].apply(process)  # 'prompt'列の各要素にprocess関数を適用します
    df['response_a'] = df['response_a'].apply(process)  # 'response_a'列の各要素にprocess関数を適用します
    df['response_b'] = df['response_b'].apply(process)  # 'response_b'列の各要素にprocess関数を適用します
    
    # テキストデータを結合します
    # df['combined_text'] = '[PROMPT] ' + df['prompt'] + ' [RESPONSE_A] ' + df['response_a'] + ' [RESPONSE_B] ' + df['response_b']  # 結合済みテキストを新しい列に格納するためのコメントがあります

In [ ]:
combine_text(train)  # trainデータフレームに対してcombine_text関数を呼び出して、テキストを結合します

# print(train['combined_text'][69])  # 結合されたテキストの69番目の要素を表示するためのコメントがあります（現在はコメントアウトされています）

In [ ]:
# ラベルを作成します
def create_label(df):
    def process(row):
        # model_aが勝者の場合、ラベル0を返します
        if row['winner_model_a'] == 1:
            return 0
        # model_bが勝者の場合、ラベル1を返します
        elif row['winner_model_b'] == 1:
            return 1
        # 引き分けの場合、ラベル2を返します
        elif row['winner_tie'] == 1:
            return 2
        
    # 各行に対してprocess関数を適用し、新しい'label'列を作成します
    df['label'] = df.apply(process, axis=1)  # axis=1は行ごとに処理することを示しています

In [ ]:
create_label(train)  # trainデータフレームに対してcreate_label関数を呼び出して、ラベルを作成します
print(train['label'][69])  # 作成されたラベルの69番目の要素を表示します

In [ ]:
# trainデータフレームの形状を表示します
print("train.shape", train.shape)  # 行数と列数を出力します

# trainデータフレームの最初の5行を表示します
train.head()  # データの最初の5行を確認して、内容を表示します

## トークナイザー

In [ ]:
# トークンの最大長を設定します
max_length = 1024  # モデルに入力できるトークンの最大数を1024に設定します

In [ ]:
# DebertaV2Tokenizerを使用したトークナイゼーション
model_name = "/kaggle/input/qwen2/transformers/qwen2-7b-instruct/1"  # 使用するモデルのパスを指定します
# model_name = "/kaggle/input/deberta-v3/pytorch/large/1"  # 別のモデル名をコメントアウトしています
tokenizer = AutoTokenizer.from_pretrained(model_name)  # 指定したモデルからトークナイザーをロードします
# 語彙のサイズを制限します
# tokenizer.model_max_length = max_length  # 最大トークン長を設定するコメントがあります
tokenizer.add_tokens(['[CLS]', '[SEP]', '[PAD]'], special_tokens=True)  # 特殊トークンを追加します

In [ ]:
def tokenize_df(df, tokenizer):
    # 特殊トークンが存在しない場合、チェックして設定します
    if tokenizer.cls_token_id is None:
        tokenizer.cls_token_id = tokenizer.convert_tokens_to_ids('[CLS]')  # '[CLS]'トークンのIDを設定します
    if tokenizer.sep_token_id is None:
        tokenizer.sep_token_id = tokenizer.convert_tokens_to_ids('[SEP]')  # '[SEP]'トークンのIDを設定します
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('[PAD]')  # '[PAD]'トークンのIDを設定します
        
    def process(row):
        max_len = max_length - 2  # セパレータトークン2つ分を考慮した最大長を計算します
        # プロンプトをトークナイズします
        prompt_tokens = tokenizer(row['prompt'], truncation=True, max_length=max_len//4)['input_ids']
        remaining_length = max_len - len(prompt_tokens)  # 残りの長さを計算します

        # レスポンスAをトークナイズします
        response_a_tokens = tokenizer(row['response_a'], truncation=True, max_length=remaining_length//2)['input_ids']
        remaining_length -= len(response_a_tokens)  # 残りの長さを更新します

        # レスポンスBをトークナイズします
        response_b_tokens = tokenizer(row['response_b'], truncation=True, max_length=remaining_length//2)['input_ids']

        # トークンを追加します
        input_ids = [tokenizer.cls_token_id] + prompt_tokens + [tokenizer.sep_token_id] + response_a_tokens + [tokenizer.sep_token_id] + response_b_tokens
        token_type_ids = [0] * (len(prompt_tokens) + 2) + [1] * (len(response_a_tokens) + 1) + [2] * len(response_b_tokens)
        attention_mask = [1] * len(input_ids)

        # パディングを追加します
        padding_length = max_length - len(input_ids)
        if padding_length > 0:
            input_ids = input_ids + [tokenizer.pad_token_id] * padding_length  # パディングトークンで埋めます
            token_type_ids = token_type_ids + [0] * padding_length
            attention_mask = attention_mask + [0] * padding_length

        input_ids = input_ids[:max_length]  # 最大長を超える場合は切り詰めます
        token_type_ids = token_type_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        
        return input_ids, token_type_ids, attention_mask
    
    # 各行に対してprocess関数を適用し、新しいトークン関連の列を作成します
    df[['input_ids', 'token_type_ids', 'attention_mask']] = df.apply(lambda row: pd.Series(process(row)), axis=1)
#     tokenized = df.apply(lambda row: pd.Series(process(row)), axis=1)  # コメントアウトされたコード
#     df.loc[:, ['input_ids', 'token_type_ids', 'attention_mask']] = tokenized  # コメントアウトされたコード
#     return df  # コメントアウトされたコード

In [ ]:
# ラベルをカテゴリカル形式に変換します
labels = tf.keras.utils.to_categorical(train['label'], num_classes=3)  # 3つのクラスでカテゴリカルラベルを生成します

In [ ]:
# trainデータフレームに対して、トークナイゼーションを実施します
tokenize_df(train, tokenizer)  # tokenize_df関数を呼び出して、トークン関連の列を生成します

In [ ]:
# トレーニングのためのデータを準備します
input_ids = train['input_ids']  # 'input_ids'列を取得します
attention_mask = train['attention_mask']  # 'attention_mask'列を取得します

# input_idsとattention_maskを最大長に合わせてパディングします
X_train = sq.pad_sequences(input_ids, maxlen=max_length)  # 入力IDにパディングを適用します
X_train_attention_mask = sq.pad_sequences(attention_mask, maxlen=max_length)  # 注意マスクにパディングを適用します

y_train = labels  # ラベルをy_trainに格納します

## モデル

In [ ]:
from keras.layers import concatenate, Dropout, BatchNormalization, LSTM, Conv1D, Masking  # 必要なKerasレイヤーをインポートします

# CNNモデルを定義します
def create_cnn_model(vocab_size, embedding_dim, max_length):
    model = Sequential([
        Input(shape=(max_length,), dtype=tf.int32, name='input_ids'),  # 入力の形状を指定します
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),  # 埋め込み層を追加します
        Conv1D(filters=256, kernel_size=5, activation='relu'),  # 畳み込み層を追加します
        GlobalMaxPooling1D(),  # グローバルマックスプーリング層を追加します
        Dense(128, activation='relu'),  # 全結合層を追加します
        Dropout(0.5),  # ドロップアウト層を追加して過学習を防ぎます
        Dense(3, activation='softmax')  # 最終出力層、3クラスのソフトマックス出力を追加します
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # モデルをコンパイルします
    return model

# LSTMモデルを定義します
def create_lstm_model(vocab_size, embedding_dim, max_length):
    model = Sequential([
        Input(shape=(max_length,), dtype=tf.int32, name='input_ids'),  # 入力の形状を指定します
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),  # 埋め込み層を追加します
        LSTM(256, return_sequences=True),  # LSTM層を追加します
        GlobalMaxPooling1D(),  # グローバルマックスプーリング層を追加します
        Dense(128, activation='relu'),  # 全結合層を追加します
        Dropout(0.5),  # ドロップアウト層を追加します
        Dense(3, activation='softmax')  # 最終出力層、3クラスのソフトマックス出力を追加します
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # モデルをコンパイルします
    return model

# CNN LSTMモデルを定義します
def create_cnn_lstm_model(vocab_size, embedding_dim, max_length):
    model = Sequential([
        Input(shape=(max_length,), dtype=tf.int32, name='input_ids'),  # 入力の形状を指定します
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),  # 埋め込み層を追加します
        Conv1D(filters=128, kernel_size=5, activation='relu'),  # 畳み込み層を追加します
        LSTM(128, return_sequences=True),  # LSTM層を追加します
        GlobalMaxPooling1D(),  # グローバルマックスプーリング層を追加します
        Dense(64, activation='relu'),  # 全結合層を追加します
        Dropout(0.5),  # ドロップアウト層を追加して過学習を防ぎます
        Dense(3, activation='softmax')  # 最終出力層、3クラスのソフトマックス出力を追加します
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # モデルをコンパイルします
    return model

In [ ]:
# パラメータを設定します
vocab_size = tokenizer.vocab_size  # トークナイザーから語彙サイズを取得します
# vocab_size = max_length  # コメントアウトされたコード
embedding_dim = 100  # 埋め込み次元を設定します
max_length = max_length  # 最大長を設定します
max_features = tokenizer.vocab_size  # 最大特徴量を語彙サイズで設定します
# max_features = max_length * 2  # コメントアウトされたコード
max_len = max_length  # 最大長をmax_lenに再設定します
maxlen = max_len  # maxlenにその値を設定します
batch_size = 16  # バッチサイズを設定します
embedding_dims = 100  # 埋め込み次元を設定します
nb_filter = 150  # フィルター数を設定します
filter_length = 3  # フィルターの長さを設定します
hidden_dims = 100  # 隠れ層の次元を設定します
nb_epoch = 100  # エポック数を設定します

# モデルを作成します
# model = create_lstm_model(vocab_size, embedding_dim, max_length)  # LSTMモデルを作成するためのコメントアウトされたコード
model = create_cnn_lstm_model(vocab_size, embedding_dim, max_length)  # CNN LSTMモデルを作成します
model.summary()  # モデルの要約を表示します

In [ ]:
from __future__ import print_function  # 将来のバージョンとの互換性のためのインポート
import numpy as np  # 数値計算のためのnumpyライブラリをインポートします

from keras.preprocessing import sequence  # シーケンス処理のためのKerasのpreprocessingモジュールをインポートします
from keras.models import Sequential  # Kerasの順次モデルをインポートします
from keras.layers import Dense, Dropout, Activation, Lambda  # 様々なKerasレイヤーをインポートします
from keras.layers import Embedding  # 埋め込み層をインポートします
from keras.layers import Convolution1D, LSTM  # 1次元畳み込み層とLSTMをインポートします
from keras.datasets import imdb  # IMDbデータセットをインポートします
from keras import backend as K  # Kerasバックエンドをインポートします
from keras.optimizers import Adadelta, Adamax  # 最適化アルゴリズムをインポートします
from keras.preprocessing import sequence as sq  # シーケンス処理のためのKerasのpreprocessingモジュールを再インポートします

from keras.layers import Dense, Dropout, Activation, Lambda, Input, TimeDistributed, Flatten  # 複数のKerasレイヤーをインポートします
from keras.models import Model  # Kerasのモデルクラスをインポートします
from keras.callbacks import ModelCheckpoint  # モデルチェックポイント用のコールバックをインポートします

In [ ]:
# 使用されていないインポートのコメントアウトされたコード
# from tensorflow.keras.layers import Layer  # Kerasレイヤーをインポートします
# from keras.layers import concatenate, Dropout, BatchNormalization, LSTM, Conv1D  # 複数のKerasレイヤーをインポートします
# from keras.layers import GlobalMaxPooling1D  # グローバルマックスプーリング層をインポートします
# import tensorflow as tf  # TensorFlowライブラリをインポートします

# ApplyAttentionMaskクラスを定義します
# class ApplyAttentionMask(Layer):
#     def call(self, inputs):
#         embeddings, attention_mask = inputs  # 入力の分離
#         return embeddings * tf.expand_dims(attention_mask, -1)  # アテンションマスクを適用します

# 入力層を定義します
# input_layer = Input(shape=(max_length,), dtype='int32', name='main_input')  # 主入力層を定義します
# attention_masks = Input(shape=(max_length,), dtype='float32', name="attention_masks")  # アテンションマスク用の入力層を定義します

# 埋め込み層を定義します
# emb_layer = Embedding(max_features,
#                       embedding_dims,
#                       input_length=max_len
#                       )(input_layer)  # 埋め込み層を定義します

# アテンションマスクを適用した埋め込みを定義します
# masked_embeddings = ApplyAttentionMask(name='apply_attention_mask')([emb_layer, attention_masks])

# LSTMブランチを定義します
# lstm_out = LSTM(128, return_sequences=True)(masked_embeddings)  # 最初のLSTM層を追加します
# lstm_out = LSTM(64, return_sequences=True)(lstm_out)  # 2つ目のLSTM層を追加します
# lstm_out = LSTM(32)(lstm_out)  # 3つ目のLSTM層を追加します
# lstm_out = BatchNormalization()(lstm_out)  # バッチ正規化を適用します
# lstm_out = Dropout(0.5)(lstm_out)  # ドロップアウトを適用します
# lstm_out = GlobalMaxPooling1D()(lstm_out)  # グローバルマックスプーリングを適用します

# CNN層ブランチを定義します
# cnn_out = Conv1D(128, 5, activation='relu')(masked_embeddings)  # 畳み込み層を追加します
# cnn_out = Conv1D(64, 5, activation='relu')(cnn_out)  # 2つ目の畳み込み層を追加します
# cnn_out = Conv1D(32, 5, activation='relu')(cnn_out)  # 3つ目の畳み込み層を追加します
# cnn_out = BatchNormalization()(cnn_out)  # バッチ正規化を適用します
# cnn_out = Dropout(0.5)(cnn_out)  # ドロップアウトを適用します
# cnn_out = GlobalMaxPooling1D()(cnn_out)  # グローバルマックスプーリングを適用します

# LSTMとCNNの出力を連結します
# merged = concatenate([lstm_out, cnn_out])  # 出力を連結します
# merged = Dense(32, activation='sigmoid')(merged)  # 全結合層を追加します
# merged = BatchNormalization()(merged)  # バッチ正規化を適用します
# merged = Dropout(0.5)(merged)  # ドロップアウトを適用します
# pred = Dense(3, activation='softmax')(merged)  # 最終出力層を追加します

# モデルを構築します
# model = Model(inputs=[input_layer, attention_masks], outputs=[pred])  # モデルを構築します
# adadelta = Adadelta(learning_rate=1.0, rho=0.75, epsilon=1e-06)  # Adadeltaオプティマイザの設定です
# adamax = Adamax(learning_rate=0.001)  # Adamaxオプティマイザの設定です
# model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])  # モデルをコンパイルします
# model.summary()  # モデルの要約を表示します

In [ ]:
# import tensorflow as tf  # TensorFlowライブラリをインポートします
# from tensorflow.keras.layers import Input, Conv1D, LSTM, GRU, Dense, Masking  # Kerasレイヤーをインポートします
# from tensorflow.keras.models import Model  # Kerasのモデルクラスをインポートします
# from transformers import DebertaTokenizer, AutoModel  # Hugging Faceのトランスフォーマーモデルをインポートします

# ハイブリッドCNN-LSTMモデルを作成する関数を定義します
# def create_cnn_lstm_hybrid_model(base_model_name, cnn_output_channels, cnn_kernel_size, hidden_dim, num_classes):
#     # 事前学習済みBERTモデルをロードします
#     model = AutoModel.from_pretrained(base_model_name)  # 指定されたモデル名からモデルをロードします
    
#     # 入力を定義します
#     input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')  # input_ids用の入力層
#     attention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')  # attention_mask用の入力層

#     # BERTの出力を取得します
#     outputs = model(input_ids, attention_mask=attention_mask)  # モデルによって出力を取得します
#     seq_output = outputs.last_hidden_state  # seq_outputの形状: (バッチサイズ, シーケンス長, 隠れサイズ)

#     # CNNを適用します
#     cnn_output = Conv1D(filters=cnn_output_channels, kernel_size=cnn_kernel_size, padding='same', activation='relu')(seq_output)  # 畳み込み層を適用します

#     # パディングされたシーケンスを処理するためにマスキングを適用します
#     masked_cnn_output = Masking()(cnn_output)  # マスキングを適用します
    
#     # LSTMを適用します
#     rnn_output = LSTM(hidden_dim)(masked_cnn_output)  # LSTM層を適用します

#     # クラス分類用の層を定義します
#     logits = Dense(num_classes, activation='softmax')(rnn_output)  # クラス数に応じた出力層を設定します

#     # モデルを作成します
#     model = Model(inputs=[input_ids, attention_mask], outputs=logits)  # モデルを構築します

#     return model  # モデルを返します

In [ ]:
# ハイパーパラメータを定義します
# bert_model_name = '/kaggle/input/deberta_v3/keras/deberta_v3_large_en/2'  # 使用するBERTモデルのパス
# bert_model_name = '/kaggle/input/deberta-v3/pytorch/large/1'  # 別のBERTモデルのパス
# bert_model_name = 'deberta_v3_large_en'  # 使用するBERTモデル名
# model_name = '/kaggle/input/qwen2/transformers/qwen2-7b-instruct/1'  # 使用するQwenモデルのパス
# cnn_output_channels = 128  # CNNの出力チャンネル数
# cnn_kernel_size = 5  # CNNのカーネルサイズ
# hidden_dim = 256  # 隠れ層の次元
# num_classes = 3  # クラス数

# # モデルを初期化します
# model = create_cnn_lstm_hybrid_model(model_name, cnn_output_channels, cnn_kernel_size, hidden_dim, num_classes)  # ハイブリッドモデルを作成します
# model.summary()  # モデルの要約を表示します

In [ ]:
from keras.callbacks import EarlyStopping  # 早期終了用のコールバックをインポートします

# モデルをトレーニングします
early_stopping = EarlyStopping(monitor='val_loss', patience=8, verbose=1)  # バリデーション損失を監視し、8エポックで改善がない場合にトレーニングを停止します

# モデルのフィッティングを行います
history = model.fit([X_train, X_train_attention_mask], y_train, epochs=20, batch_size=32, validation_split=0.2,  # バリデーションデータを分割して使用します
                    callbacks=[early_stopping])  # 早期終了のコールバックを適用します

## テスト

In [ ]:
# テストデータをエンコードします
combine_text(test)  # テストデータに対してテキストを結合します
tokenize_df(test, tokenizer)  # テストデータに対してトークナイゼーションを実施します

input_ids = test['input_ids']  # 'input_ids'列を取得します
attention_mask = test['attention_mask']  # 'attention_mask'列を取得します

# input_idsとattention_maskを最大長に合わせてパディングします
X_test = sq.pad_sequences(input_ids, maxlen=max_length)  # テストデータの入力IDにパディングを適用します
X_test_attention_mask = sq.pad_sequences(attention_mask, maxlen=max_length)  # テストデータの注意マスクにパディングを適用します

In [ ]:
# テストデータに対して予測を行います
predictions = model.predict([X_test, X_test_attention_mask])  # モデルを使用して予測を行います
predictions  # 予測結果を表示します

In [ ]:
# 予測結果をデータフレームに変換します
winner = pd.DataFrame(predictions, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])  # 予測結果をデータフレームに格納します
result = pd.concat([test['id'], winner], axis=1)  # テストデータのIDと予測結果を結合します

# 結果をCSVファイルに保存します
result.to_csv('submission.csv', index=False)  # 提出用のCSVファイルを作成します
result  # 結果を表示します